In [101]:
import cv2
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylsd.lsd import lsd
%matplotlib inline

In [102]:
img_path = "/Users/Chihiro/Personal/10_Projects/GPC/get_plant_plate_txt/image/20200411_1.jpg"
output_dir = "/Users/Chihiro/Desktop/"

In [187]:
def mask_img(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
# #     ハフ変換
#     lines = cv2.HoughLines(img_gray, 
#                             rho=1, 
#                             theta=np.pi/360, 
#                             threshold=4450)
#     print(len(lines))
    
#     # 検出した線の描画
#     for line in lines:
#         for rho, theta in line:
#             a = np.cos(theta)
#             b = np.sin(theta)
#             x0 = a*rho
#             y0 = b*rho
#             x1 = int(x0 + 1000*(-b))
#             y1 = int(y0 + 1000*(a))
#             x2 = int(x0 - 1000*(-b))
#             y2 = int(y0 - 1000*(a))

#             cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
#     cv2.imwrite(output_dir + "lines.png", img)

    img_g = cv2.GaussianBlur(img_gray,(5,5),5)
    img_g = cv2.Canny(img_g,50,150,apertureSize = 3)
    linesH = cv2.HoughLinesP(img_g, rho=1, theta=np.pi/360, threshold=100, minLineLength=100, maxLineGap=1)
    
    for line in linesH:
        x1, y1, x2, y2 = line[0]

        # 赤線を引く
        img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 10)


#     LSD
    lines = lsd(img_gray)
#     for line in lines:
#         x1, y1, x2, y2 = map(int,line[:4])
#         img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 3)
#         if (x2-x1)**2 + (y2-y1)**2 > 1000:
#            # 赤線を引く
#            img = cv2.line(img, (x1,y1), (x2,y2), (0,0,255), 3)
            
            
    cv2.imwrite(output_dir + "lines.png", img) 

In [188]:
def detect_contour(path):
    img = cv2.imread(img_path)
    masked_img = mask_img(img)

In [189]:
detect_contour(img_path)